# Import Event Log

In [46]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the XES file
    dataframe_log = pm4py.read_xes('../../data/logs/RequestForPayment.xes')

    # If 'log' is already a DataFrame, add the @@case_index column directly
    case_indices = {case_id: idx for idx, case_id in enumerate(dataframe_log['case:concept:name'].unique())}
    dataframe_log['@@case_index'] = dataframe_log['case:concept:name'].map(case_indices)
    
     # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

parsing log, completed traces :: 100%|██████████| 6886/6886 [00:14<00:00, 468.34it/s] 


,id,org:resource,concept:name,time:timestamp,org:role,case:Rfp_id,case:Project,case:Task,case:concept:name,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber,@@case_index
0,st_step 148220_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-01-09 09:17:18+00:00,EMPLOYEE,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
1,st_step 148221_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2017-01-09 09:18:00+00:00,SUPERVISOR,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
2,st_step 148222_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,2017-01-10 12:42:32+00:00,MISSING,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
3,st_step 148219_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-03-03 09:51:13+00:00,EMPLOYEE,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
4,st_step 148218_0,STAFF MEMBER,Request For Payment APPROVED by PRE_APPROVER,2017-03-03 09:51:42+00:00,PRE_APPROVER,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36791,st_step 185004_0,STAFF MEMBER,Request For Payment APPROVED by ADMINISTRATION,2018-12-29 12:35:02+00:00,ADMINISTRATION,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001,6885
36792,st_step 185003_0,STAFF MEMBER,Request For Payment APPROVED by BUDGET OWNER,2019-01-03 09:27:20+00:00,BUDGET OWNER,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001,6885
36793,st_step 185005_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2019-01-08 09:00:39+00:00,SUPERVISOR,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001,6885
36794,rp_request for payment 185000_15,SYSTEM,Request Payment,2019-01-08 09:29:14+00:00,UNDEFINED,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001,6885


# Drop unnessary columns

In [47]:
dataframe_log = dataframe_log.drop(columns=['case:Rfp_id'])
dataframe_log = dataframe_log.drop(columns=['case:Project'])
dataframe_log = dataframe_log.drop(columns=['case:Task'])
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])
dataframe_log = dataframe_log.drop(columns=['case:OrganizationalEntity'])
dataframe_log = dataframe_log.drop(columns=['case:Cost Type'])
dataframe_log = dataframe_log.drop(columns=['case:RequestedAmount'])
dataframe_log = dataframe_log.drop(columns=['case:Activity'])
dataframe_log = dataframe_log.drop(columns=['case:RfpNumber'])

In [48]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

# Preprocess

In [49]:
codes, uniques = pd.factorize(dataframe_log['id'])
dataframe_log['id'] = codes

In [50]:
codes, uniques = pd.factorize(dataframe_log['org:resource'])
dataframe_log['org:resource'] = codes + 1

In [51]:
codes, uniques = pd.factorize(dataframe_log['concept:name'])
dataframe_log['concept:name'] = codes + 1

In [52]:
codes, uniques = pd.factorize(dataframe_log['org:role'])
dataframe_log['org:role'] = codes + 1

# Generate Prefixes

In [53]:
df_id = dataframe_log[['id', '@@case_index']]
df_resource = dataframe_log[['org:resource', '@@case_index']]
df_activity = dataframe_log[['concept:name', '@@case_index']]
df_role = dataframe_log[['org:role', '@@case_index']]
df_timestamps = dataframe_log[['standardized_elapsed_time', '@@case_index']]

In [54]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    case_indices = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
            case_indices.append(case_id)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    case_indices_array = np.array(case_indices)
    
    return windows_padded, targets_array, case_indices_array

In [55]:
windows_id, targets_id, case_indices = generate_prefix_windows(df_id)
windows_resource, targets_resource, case_indices = generate_prefix_windows(df_resource)
windows_activity, targets_activity, case_indices = generate_prefix_windows(df_activity)
windows_role, targets_role, case_indices = generate_prefix_windows(df_role)
windows_timestamps, targets_timestamps, case_indices = generate_prefix_windows(df_timestamps)

# GRU

### Architecture

In [56]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('@@case_index').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, Concatenate, BatchNormalization

def create_binet_v4(num_ids, num_resources, num_activities, num_roles, embedding_dim, gru_units, dropout_rate):
    # Input layers for each attribute
    input_id = Input(shape=(None,), name='id_input')
    input_resource = Input(shape=(None,), name='resource_input')
    input_activity = Input(shape=(None,), name='activity_input')
    input_role = Input(shape=(None,), name='role_input')
    input_timestamp = Input(shape=(None, 1), name='timestamp_input')  # Continuous input

    # Embedding layers for categorical attributes
    embedding_id = Embedding(input_dim=num_ids + 1, output_dim=embedding_dim, mask_zero=True, name='id_embedding')(input_id)
    embedding_resource = Embedding(input_dim=num_resources + 1, output_dim=embedding_dim, mask_zero=True, name='resource_embedding')(input_resource)
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)
    embedding_role = Embedding(input_dim=num_roles + 1, output_dim=embedding_dim, mask_zero=True, name='role_embedding')(input_role)

    # Encoder GRUs with Batch Normalization for categorical attributes
    encoded_id = GRU(units=gru_units, return_sequences=True, name='id_encoder')(embedding_id)
    bn_id = BatchNormalization(name='bn_id')(encoded_id)
    encoded_resource = GRU(units=gru_units, return_sequences=True, name='resource_encoder')(embedding_resource)
    bn_resource = BatchNormalization(name='bn_resource')(encoded_resource)
    encoded_activity = GRU(units=gru_units, return_sequences=True, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)
    encoded_role = GRU(units=gru_units, return_sequences=True, name='role_encoder')(embedding_role)
    bn_role = BatchNormalization(name='bn_role')(encoded_role)

    # Encoder GRU with Batch Normalization for continuous attribute
    encoded_timestamp = GRU(units=gru_units, return_sequences=True, name='timestamp_encoder')(input_timestamp)
    bn_timestamp = BatchNormalization(name='bn_timestamp')(encoded_timestamp)

    # Concatenation of encoded outputs
    concatenated = Concatenate(name='concatenate_encodings')([bn_id, bn_resource, bn_activity, bn_role, bn_timestamp])

    # Decoder GRU
    decoder_output = GRU(units=gru_units, return_sequences=False, name='decoder_gru')(concatenated)
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(decoder_output)

    # Output layers for predicting the next event's attributes
    output_id = Dense(num_ids + 1, activation='softmax', name='output_id')(dropout_layer)
    output_resource = Dense(num_resources + 1, activation='softmax', name='output_resource')(dropout_layer)
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)
    output_role = Dense(num_roles + 1, activation='softmax', name='output_role')(dropout_layer)
    output_timestamp = Dense(1, activation='linear', name='output_timestamp')(dropout_layer)  # Linear activation for continuous output

    # Building the model
    model = Model(inputs=[input_id, input_resource, input_activity, input_role, input_timestamp], 
                  outputs=[output_id, output_resource, output_activity, output_role, output_timestamp])
    model.compile(
        optimizer='adam', 
        loss={
            'output_id': 'categorical_crossentropy', 
            'output_resource': 'categorical_crossentropy', 
            'output_activity': 'categorical_crossentropy', 
            'output_role': 'categorical_crossentropy', 
            'output_timestamp': 'mean_squared_error'
        },
        metrics={
            'output_id': ['accuracy'], 
            'output_resource': ['accuracy'], 
            'output_activity': ['accuracy'], 
            'output_role': ['accuracy'], 
            'output_timestamp': ['mean_squared_error']
        }
    )

    return model

# Parameters
gru_units = int(2 * E)
num_ids = dataframe_log['id'].nunique()
num_resources = dataframe_log['org:resource'].nunique()
num_activities = dataframe_log['concept:name'].nunique()
num_roles = dataframe_log['org:role'].nunique()
embedding_dim = 50
dropout_rate = 0.2
model = create_binet_v4(num_ids, num_resources, num_activities, num_roles, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 id_input (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 resource_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 activity_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                            

### Data Splitting

In [58]:
from sklearn.model_selection import train_test_split

# Splitting the dataset for the new attributes
train_id, test_id, train_targets_id, test_targets_id = train_test_split(
    windows_id, targets_id, test_size=0.3, random_state=42)

train_resource, test_resource, train_targets_resource, test_targets_resource = train_test_split(
    windows_resource, targets_resource, test_size=0.3, random_state=42)

train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

train_role, test_role, train_targets_role, test_targets_role = train_test_split(
    windows_role, targets_role, test_size=0.3, random_state=42)

train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamps, targets_timestamps, test_size=0.3, random_state=42)

### Training

In [59]:
from tensorflow.keras.utils import to_categorical

# Convert categorical targets to one-hot encoded format
train_targets_id_cat = to_categorical(train_targets_id, num_classes=num_ids + 1)
test_targets_id_cat = to_categorical(test_targets_id, num_classes=num_ids + 1)

train_targets_resource_cat = to_categorical(train_targets_resource, num_classes=num_resources + 1)
test_targets_resource_cat = to_categorical(test_targets_resource, num_classes=num_resources + 1)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities + 1)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities + 1)

train_targets_role_cat = to_categorical(train_targets_role, num_classes=num_roles + 1)
test_targets_role_cat = to_categorical(test_targets_role, num_classes=num_roles + 1)

In [60]:
# Fit the model with the properly shaped targets
history = model.fit(
    [train_id, train_resource, train_activity, train_role, train_timestamp],
    [train_targets_id_cat, train_targets_resource_cat, train_targets_activity_cat, train_targets_role_cat, train_targets_timestamp],
    validation_data=(
        [test_id, test_resource, test_activity, test_role, test_timestamp],
        [test_targets_id_cat, test_targets_resource_cat, test_targets_activity_cat, test_targets_role_cat, test_targets_timestamp]
    ),
    epochs=20,
    batch_size=500
)

Epoch 1/20
42/42 [==============================] - 185s 2s/step - loss: 14.6928 - output_id_loss: 10.5544 - output_resource_loss: 0.3595 - output_activity_loss: 1.8772 - output_role_loss: 1.0894 - output_timestamp_loss: 0.8123 - output_id_accuracy: 0.0000e+00 - output_resource_accuracy: 0.9152 - output_activity_accuracy: 0.5879 - output_role_accuracy: 0.7339 - output_timestamp_mean_squared_error: 0.8123 - val_loss: 16.8836 - val_output_id_loss: 10.5477 - val_output_resource_loss: 0.8316 - val_output_activity_loss: 2.6393 - val_output_role_loss: 1.7749 - val_output_timestamp_loss: 1.0901 - val_output_id_accuracy: 0.0000e+00 - val_output_resource_accuracy: 0.9853 - val_output_activity_accuracy: 0.6201 - val_output_role_accuracy: 0.6333 - val_output_timestamp_mean_squared_error: 1.0901
Epoch 2/20
42/42 [==============================] - 62s 1s/step - loss: 12.3135 - output_id_loss: 10.3096 - output_resource_loss: 0.0722 - output_activity_loss: 0.7868 - output_role_loss: 0.4883 - output_t

In [61]:
# Evaluate the model on the validation set
evaluation = model.evaluate(
    [test_id, test_resource, test_activity, test_role, test_timestamp],
    [test_targets_id_cat, test_targets_resource_cat, test_targets_activity_cat, test_targets_role_cat, test_targets_timestamp],
    batch_size=64
)

print(evaluation)

141/141 [==============================] - 9s 59ms/step - loss: 16.1122 - output_id_loss: 13.8672 - output_resource_loss: 0.0174 - output_activity_loss: 0.9746 - output_role_loss: 0.7316 - output_timestamp_loss: 0.5214 - output_id_accuracy: 0.0000e+00 - output_resource_accuracy: 0.9958 - output_activity_accuracy: 0.8024 - output_role_accuracy: 0.8565 - output_timestamp_mean_squared_error: 0.5214
[16.112157821655273, 13.86721134185791, 0.017396194860339165, 0.9745550751686096, 0.7316249012947083, 0.5213640928268433, 0.0, 0.9957650899887085, 0.8024072051048279, 0.8564582467079163, 0.5213640928268433]


# Anomaly Score Computation

In [62]:
# Generate predictions for all inputs
predictions = model.predict([windows_id, windows_resource, windows_activity, windows_role, windows_timestamps])

# Extract predictions for each attribute
# Assuming the order of outputs in the model is: id, resource, activity, role, timestamp
predictions_id = predictions[0]        # ID predictions (categorical)
predictions_resource = predictions[1]  # Resource predictions (categorical)
predictions_activity = predictions[2]  # Activity predictions (categorical)
predictions_role = predictions[3]      # Role predictions (categorical)
predictions_timestamp = predictions[4] # Timestamp predictions (continuous)

935/935 [==============================] - 56s 39ms/step


In [63]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [64]:
anomaly_scores_resource = calculate_anomaly_scores(predictions_resource, targets_resource.astype(int))
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity.astype(int))
anomaly_scores_role = calculate_anomaly_scores(predictions_role, targets_role.astype(int))

In [65]:
def compute_anomaly_scores_continuous(predictions, actuals):
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize to [0, 1] range
    max_diff = np.max(differences)
    normalized_scores = differences / max_diff if max_diff != 0 else differences
    
    return normalized_scores

In [66]:
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamps)
anomaly_scores_timestamp = anomaly_scores_timestamp.flatten()
anomaly_scores_timestamp = anomaly_scores_timestamp.tolist()

## Insert missing scores for cases with less than 2 Events

In [67]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
score = pd.DataFrame({'case': case_indices})
score['score_resource'] = anomaly_scores_resource
score['score_activity'] = anomaly_scores_activity
score['score_role'] = anomaly_scores_role
score['score_timestamp'] = anomaly_scores_timestamp

score['case'] = score['case'].astype(int)

score

,case,score_resource,score_activity,score_role,score_timestamp
0,0,0.0,0.170293,0.251040,0.030422
1,0,0.0,0.000000,0.000000,0.016085
2,0,0.0,0.000000,0.000000,0.000827
3,0,0.0,0.885653,0.920798,0.046125
4,0,0.0,0.000000,0.000000,0.010293
...,...,...,...,...,...
29905,6885,0.0,0.000000,0.000000,0.003686
29906,6885,0.0,0.000000,0.000000,0.001758
29907,6885,0.0,0.000000,0.000000,0.009988
29908,6885,0.0,0.000000,0.000000,0.006801


In [68]:
import pandas as pd

def contains_all_values(df, column, end):

    # Generate the set of all values in the specified range
    required_values = set(range(0, end + 1))
    
    # Get the unique values in the specified column
    column_values = set(df[column].unique())
    
    # Find missing values
    missing_values = required_values - column_values
    
    # Print missing values if any
    if missing_values:
        print(f"Missing values: {sorted(missing_values)}")
    
    # Check if all required values are in the column values
    return required_values.issubset(column_values)

end = 4999

result = contains_all_values(score, 'case', end)
print(f"Does the 'case' column contain all values between 0 and {end}? {result}")

Missing values: [179, 181, 188, 354, 387, 450, 529, 545, 601, 765, 835, 974, 1020, 1202, 1273, 1299, 1301, 1324, 1329, 1347, 1390, 1431, 1578, 1601, 1636, 1895, 2022, 2091, 2151, 2152, 2226, 2367, 2466, 2712, 2897, 2983, 3041, 3297, 3440, 3478, 3573, 3652, 3659, 3865, 3960, 4064, 4235, 4279, 4389, 4638, 4865]
Does the 'case' column contain all values between 0 and 4999? False


In [69]:
# Generate the full range of values
full_range = set(range(6886))

# Get the existing 'case' values
existing_cases = set(score['case'])

# Identify missing values
missing_cases = full_range - existing_cases

# Create a DataFrame for missing values
missing_df = pd.DataFrame({'case': list(missing_cases)})

# Add other columns with value 1
for col in score.columns:
    if col != 'case':
        missing_df[col] = 1

# Concatenate the original DataFrame with the missing values DataFrame
updated_score = pd.concat([score, missing_df])

# Sort the DataFrame by 'case'
updated_score = updated_score.sort_values(by='case').reset_index(drop=True)

updated_score

,case,score_resource,score_activity,score_role,score_timestamp
0,0,0.0,0.170293,0.251040,0.030422
1,0,0.0,0.000000,0.000000,0.016085
2,0,0.0,0.000000,0.000000,0.000827
3,0,0.0,0.885653,0.920798,0.046125
4,0,0.0,0.000000,0.000000,0.010293
...,...,...,...,...,...
29977,6885,0.0,0.000000,0.000000,0.003686
29978,6885,0.0,0.000000,0.000000,0.001758
29979,6885,0.0,0.000000,0.000000,0.009988
29980,6885,0.0,0.000000,0.000000,0.006801


In [70]:
anomaly_scores_resource = updated_score['score_resource']
anomaly_scores_activity = updated_score['score_activity']
anomaly_scores_role = updated_score['score_role']
anomaly_scores_timestamp = updated_score['score_timestamp']

### Threshold (lowest plateau)

In [71]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    
    # Remove duplicate values
    unique_thresholds, unique_indices = np.unique(sorted_scores, return_index=True)
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in unique_thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(unique_thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(unique_thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = unique_thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        percentile_90 = np.percentile(sorted_scores, 90)
        if percentile_90 == 1.0:
            return 0.4
        else:
            return percentile_90

In [72]:
threshold_activity = find_plateaus(anomaly_scores_activity)
threshold_resource = find_plateaus(anomaly_scores_resource)
threshold_role = find_plateaus(anomaly_scores_role)
threshold_timestamp = find_plateaus(anomaly_scores_timestamp)

In [73]:
threshold_activity

0.0

### Detection

In [74]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [75]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_resource = detect_anomalies(anomaly_scores_resource, threshold_resource)
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)
labels_role = detect_anomalies(anomaly_scores_role, threshold_role)
labels_timestamp = detect_anomalies(anomaly_scores_timestamp, threshold_timestamp)

# Mapping

In [76]:
case_indices = updated_score['case']

In [77]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_resource'] = labels_resource
mapping['predicted_activity'] = labels_activity
mapping['predicted_role'] = labels_role
mapping['predicted_timestamp'] = labels_timestamp

mapping

,case,predicted_resource,predicted_activity,predicted_role,predicted_timestamp
0,0,0,1,1,1
1,0,0,0,0,1
2,0,0,0,0,0
3,0,0,1,1,1
4,0,0,0,0,0
...,...,...,...,...,...
29977,6885,0,0,0,0
29978,6885,0,0,0,0
29979,6885,0,0,0,0
29980,6885,0,0,0,0


In [90]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_resource', 'predicted_activity', 'predicted_role', 'predicted_timestamp']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0        True
1        True
2        True
3        True
4        True
        ...  
6881     True
6882     True
6883    False
6884     True
6885    False
Length: 6886, dtype: bool

# Ground Truth

In [91]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [92]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_RequestForPayment.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants :: 100%|██████████| 89/89 [00:00<00:00, 218.27it/s]


In [93]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [94]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]

# Evaluation

In [95]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [96]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.36


In [97]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.64


In [98]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.84


In [99]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.62


In [100]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.63
